In [ ]:
import pathlib
import re
from IPython import display

import pandas

import configs2

In [ ]:
root_dir = pathlib.Path('E:\Data\WifiEnvironment\out3')
older_root_dir = pathlib.Path('E:\Data\WifiEnvironment\out2')
run_name_pattern = re.compile(r'^model_(?P<name>.*?)_room(?P<room>\d)_seed(?P<seed>\d+)(?P<time>.+)$')

models = ['base', 'bare', 'bare_margin', 'mine', 'mine_wosmoothness',
          # 'mine_grl', 'mine_grl_wosmoothness',
          'cnn', 'wicarimproved', 'ei_improved',
          'ei_base', 'ei',  # 'ei_constraint0.15',
          'wicarbase', 'wicar',]  # 'wicar_lambda0.1', 'wicar_lambda0.2', 'wicar_lambda0.05']
          # 'wicarbaselarge', 'wicarlarge', 'wicarlarge_lambda0.1', 'wicarlarge_lambda0.05', ]
seeds = configs2.model_comparison_seeds
rooms = [i for i in range(configs2.data_count['room'])]

In [ ]:
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)

In [ ]:
def get_experiment_results(directory: pathlib.Path):
    result_dict = {}
    for experiment_dir in directory.iterdir():
        if not experiment_dir.is_dir():
            continue

        experiment_name = experiment_dir.name

        run_list = []
        for run_dir in experiment_dir.iterdir():
            if not run_dir.is_dir():
                continue

            match = run_name_pattern.fullmatch(run_dir.name)
            room, seed = match.group('room', 'seed')

            validation_results = pandas.read_csv((run_dir / 'validation.csv'), sep = '\t')

            run_list.append(
                    {'room': int(room), 'seed': int(seed), 'path': run_dir.name,
                     'validation_results': validation_results})

        run_df = pandas.DataFrame(data = run_list, columns = ['room', 'seed', 'path', 'validation_results'])
        run_df = run_df.astype({'room': 'category', 'seed': 'category'})

        result: pandas.DataFrame = pandas.read_csv((experiment_dir / 'result.csv'), sep = '\t')
        result = result.astype({'room': 'category', 'seed': 'category'})
        best_result: pandas.DataFrame = pandas.read_csv((experiment_dir / 'result_best.csv'), sep = '\t')
        best_result = best_result.astype({'room': 'category', 'seed': 'category'})

        result_dict.update({experiment_name: {'final': result, 'best': best_result, 'runs': run_df}})
    return result_dict

In [ ]:
def get_average_results(results: pandas.DataFrame) -> pandas.DataFrame:
    return results.groupby(['room']).describe(include = ['float64'])


def get_average_validation_results(run_results: pandas.DataFrame, start: int, end: int) -> pandas.DataFrame:
    def func(data: pandas.Series):
        validation_results: pandas.DataFrame = data['validation_results']
        validation_results.drop(columns = ['loss_classifier', 'loss_discriminator', 'loss_total'],
                                inplace = True, errors = 'ignore')
        
        validation_results = validation_results.loc[
            (start <= validation_results['epoch']) & (validation_results['epoch'] <= end)]
        
        validation_results.insert(0, 'seed', data['seed'])
        validation_results.insert(0, 'room', data['room'])
        validation_results = validation_results.astype({'room': 'category', 'seed': 'category', 'epoch': 'category'})
        
        return validation_results
    
    results: pandas.DataFrame = run_results.apply(func, axis = 1)
    results = pandas.concat([data for _, data in results.iteritems()], ignore_index = True)
    results = results.astype({'room': 'category', 'seed': 'category', 'epoch': 'category'})
    
    return results.groupby(['room']).describe(include = ['float64'])

In [ ]:
experiment_results = get_experiment_results(root_dir)
older_experiment_results = get_experiment_results(older_root_dir)

In [ ]:
results = {}
for model_name in models:
    if model_name in experiment_results:
        model_results = experiment_results[model_name]
    elif model_name in older_experiment_results:
        model_results = older_experiment_results[model_name]
    else:
        continue

    final_result = get_average_results(model_results['final'])
    best_result = get_average_results(model_results['best'])
    last_result = get_average_validation_results(model_results['runs'], 96, 100)

    results.update({model_name: {'final': final_result, 'best': best_result, 'last': last_result}})

    print(model_name)
    display.display(last_result)

    del model_name, model_results, final_result, best_result, last_result